<a href="https://colab.research.google.com/github/gil612/NLP/blob/main/NLP_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text os speech)

Anothe common term for NLP problems is `seq2seq` problems.


# Connect to GPU

In [1]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-09-29 20:59:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-29 20:59:33 (70.7 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2023-09-29 21:00:13--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 2607:f8b0:4023:c03::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.6’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-09-29 21:00:13 (132 MB/s) - ‘nlp_getting_started.zip.6’ saved [607343/607343]



In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# New Section

In [5]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df), len(test_df)

(7613, 3263)

In [67]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [69]:
import random
random_index = random.randint(0,len(train_df) - 5)
# Create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index: random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")



Target: 0 (not real disaster)
Text:
What the hell is wrong with people?!communication!  Rule with an iron fist ticket to disaster.  Can't swing one day and not the next.

---

Target: 0 (not real disaster)
Text:
Let's destroy Twitter with @fouseyTUBE @zaynmalik come on BrBrS lets do it in morning with fousey

---

Target: 1 (real disaster)
Text:
Heroes! A Springer Spaniel &amp; her dog dad rescued a stranded baby dolphin: http://t.co/G03DkPooNP http://t.co/Go0HPi0B4c

---

Target: 0 (not real disaster)
Text:
We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw

---

Target: 0 (not real disaster)
Text:
@BabySweet420 I'm mad 420 in your name &amp; you don't blaze.

---



In [11]:
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


## Split data into training and validation sets

In [70]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Convert text into number

When dea;ingwith a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature  vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [71]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [18]:
# train_sentences[0]

twt = "@mogacola @zamtriossu i screamed after hitting tweet"
twt.split(), len(twt.split())


(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [19]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [72]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [73]:
## adapt()
# Computes a vocabulary of string terms from tokens in a dataset.

# Calling adapt() on a TextVectorization layer is an alternative to
# passing in a precomputed vocabulary on construction via the vocabulary
# argument. A TextVectorization layer should always be either adapted
# over a dataset or supplied with a vocabulary.

# During adapt(), the layer will build a vocabulary of all string tokens
# seen in the dataset, sorted by occurrence count, with ties broken by
# sort order of the tokens (high to low). At the end of adapt(), if
# max_tokens is set, the vocabulary wil be truncated to max_tokens
# size. For example, adapting a layer with max_tokens=1000 will compute
# the 1000 most frequent tokens occurring in the input dataset. If
# output_mode='tf-idf', adapt() will also learn the document
# frequencies of each token in the input dataset.

# In order to make TextVectorization efficient in any distribution
# context, the vocabulary is kept static with respect to any compiled
# tf.Graphs that call the layer. As a consequence, if the layer is
# adapted a second time, any models using the layer should be re-compiled.
# For more information see
# tf.keras.layers.experimental.preprocessing.PreprocessingLayer.adapt.

# adapt() is meant only as a single machine utility to compute layer
# state.  To analyze a dataset that cannot fit on a single machine, see
# [Tensorflow Transform](
# https://www.tensorflow.org/tfx/transform/get_started) for a
# multi-machine, map-reduce solution.

# Arguments:
#   data: The data to train on. It can be passed either as a
#       tf.data.Dataset, or as a numpy array.
#   batch_size: Integer or None.
#       Number of samples per state update.
# If unspecified, batch_size will default to 32.
# Do not specify the batch_size if your data is in the
# form of datasets, generators, or keras.utils.Sequence instances
# (since they generate batches).
#   steps: Integer or None.
#       Total number of steps (batches of samples)
# When training with input tensors such as
# TensorFlow data tensors, the default None is equal to
# the number of samples in your dataset divided by
# the batch size, or 1 if that cannot be determined. If x is a
# tf.data dataset, and 'steps' is None, the epoch will run until
# the input dataset is exhausted. When passing an infinitely
# repeating dataset, you must specify the steps argument. This
# argument is not supported with array inputs.


# fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [74]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
import random

In [75]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 790,  634,  206,  534,   87,  189,   65,  551,  108,    4, 2432,
         790,  790,  790, 2543]])>

In [76]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words in vocab: {top_5_words}")
print(f"5 least common words in vocab: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
5 least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [77]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

### Creating an Embedding using an Embedding Layer

To make out embedding, we're going to use TF's layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

Thhe parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [78]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

In [79]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00652937,  0.03770499, -0.03208432, ..., -0.01450961,
         -0.02303728, -0.02277918],
        [ 0.03115221, -0.03183019, -0.02212439, ..., -0.04207458,
         -0.01977497,  0.00359417],
        [ 0.00138425,  0.0341349 ,  0.04292533, ..., -0.04943069,
          0.03533293,  0.01011633],
        ...,
        [-0.02639557, -0.03531104, -0.00789667, ..., -0.00753368,
          0.03815465, -0.03326895],
        [-0.02639557, -0.03531104, -0.00789667, ..., -0.00753368,
          0.03815465, -0.03326895],
        [-0.02639557, -0.03531104, -0.00789667, ..., -0.00753368,
          0.03815465, -0.03326895]]], dtype=float32)>

In [29]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03495445,  0.0171347 ,  0.02517421,  0.02160709,  0.04118817,
         0.02117875, -0.00814104, -0.00643381, -0.0230618 , -0.00974107,
         0.01098762,  0.00532582,  0.01857256,  0.01976607, -0.00091678,
        -0.04119305,  0.04025269, -0.00750806,  0.03122196,  0.02955203,
        -0.03965552, -0.0135177 ,  0.03890941,  0.02111658,  0.0201521 ,
         0.00100019,  0.04191686, -0.02588482, -0.02617118,  0.01976824,
        -0.04657447, -0.02711489,  0.0126433 , -0.04145421,  0.01491029,
         0.02183086, -0.03598969, -0.01840417, -0.00760845, -0.01366639,
         0.02192609,  0.0464235 ,  0.02505025,  0.0395943 , -0.04441564,
        -0.01635652, -0.0003037 ,  0.0402838 ,  0.00894971, -0.04312335,
        -0.00919584,  0.02328761,  0.0303162 , -0.01332177,  0.02938749,
         0.03306362, -0.01058891, -0.0409992 ,  0.03727648, -0.01774651,
         0.02186393,  0.02926494, -0.02032192,  0.02870793,  0.04381831,
  

In [30]:
# Text vectorization / toeknization is converting words to some numrical format

## Modelling and text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series od modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed_forward nueral netwerok (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirictional-LSTM model (RNN)
* Modle 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (using Transfer Learning TL for NLP)
* Model 7: Same as model 6 with 10% of trainig data

How are we going to approach all of these?

Use standard steps in modelling with TF:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machimne learning modelling experiments, it's important ro create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baselin, we'l use Sklearn;s Multinomial NB using the TF-IDF formula to convert our words to numbers.

> **Note:** It's a common practice to use non-DL algorithms as baseline because of their speed and then later using their DL to see if you can improve upon them.

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [81]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [33]:
train_df.value_counts()

id     keyword   location                        text                                                                                                                                 target
48     ablaze    Birmingham                      @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C                                                                              1         1
7166   mudslide  Chiswick, London                2 great new recipes; mudslide cake and so sorry stew! #GBBO                                                                          0         1
7178   mudslide  Edinburgh                       @hazelannmac ooh now I feel guilty about wishing hatman out. I bet the mudslide was delicious!                                       0         1
7176   mudslide  London                          First impressions: glad hat man is leaving in lieu of more interesting ladies. Hope mudslide lady triumphs next week.                0         1
7174   mudslide  The Pumpkin Carria

In [82]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [83]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [84]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense Model


In [85]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [38]:
# # Build model with the Functional API
# from tensorflow.keras import layers
# inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
# x = text_vectorizer(inputs) # turn the input text into numbers
# x = embedding(x) # create an embedding of the numerized numbers
# #x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
# outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
# model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [39]:
# Get a summary of the model
# model_1.summary()

# Once it goes from a text vectorization to embedding every token, so all 15 tokens get represened as a 128 long feature vector,
# and then we pass this embedding through our final dense layer, which outputs the size of 1.

In [40]:
# # Compile model
# model_1.compile(loss="binary_crossentropy",
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=["accuracy"])

In [41]:
# # Fit the model
# model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
#                               train_labels,
#                               epochs=5,
#                               validation_data=(val_sentences, val_labels),
#                               callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
#                                                                      experiment_name="simple_dense_model")])

In [42]:
# # Check the results
# model_1.evaluate(val_sentences, val_labels)

In [43]:
# #Make some predictions and evaluate those
# model_1_pred_probs = model_1.predict(val_sentences)
# model_1_pred_probs.shape

# # It's outputing prediction probability for each token -> poor results

In [44]:
# model_1_pred_probs[0]

In [86]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [88]:
# Get a summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [87]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [89]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230929-214652
Epoch 1/5
215/215 [==============================] - 17s 71ms/step - loss: 0.6101 - accuracy: 0.6933 - val_loss: 0.5359 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4413 - accuracy: 0.8196 - val_loss: 0.4685 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3464 - accuracy: 0.8609 - val_loss: 0.4585 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2849 - accuracy: 0.8927 - val_loss: 0.4635 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2381 - accuracy: 0.9120 - val_loss: 0.4760 - val_accuracy: 0.7861


In [90]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4760 - accuracy: 0.7861


[0.47599801421165466, 0.7860892415046692]

In [91]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.01062573, -0.05275462,  0.00800254, ...,  0.00910792,
          0.02307833, -0.01772935],
        [ 0.01038432,  0.0319877 , -0.02703985, ..., -0.00974621,
         -0.02744362, -0.01787872],
        [ 0.02277392, -0.05065035,  0.00266097, ..., -0.01508311,
         -0.0459183 ,  0.03991572],
        ...,
        [-0.00785009, -0.04548398, -0.04408681, ...,  0.01241822,
         -0.02340556, -0.00083836],
        [-0.00659926, -0.06605855,  0.07089156, ...,  0.07888445,
         -0.0416574 ,  0.06723114],
        [ 0.11051007, -0.04001845,  0.10285289, ...,  0.05413506,
         -0.08002777,  0.10381921]], dtype=float32)>]

In [92]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [93]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only

24/24 [==============================] - 0s 4ms/step


array([[0.4152617 ],
       [0.7459546 ],
       [0.9975573 ],
       [0.10930366],
       [0.11574414],
       [0.93554026],
       [0.9159111 ],
       [0.99298894],
       [0.9687519 ],
       [0.2592714 ]], dtype=float32)

In [50]:
model_1_pred_probs[0]
# Anything under threshold 0.4955 is going to get label 0

array([0.48681444], dtype=float32)

In [51]:
model_1_pred_probs[:10]

array([[0.48681444],
       [0.4955394 ],
       [0.5017971 ],
       [0.4939393 ],
       [0.50066817],
       [0.49665186],
       [0.49967405],
       [0.4995479 ],
       [0.49529833],
       [0.49812216]], dtype=float32)

In [96]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [94]:
# convert model prediction probabilities tp label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [95]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [55]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [97]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualizing learned embeddings

In [98]:
# get the vocaulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [100]:
# get the weight matrix of embedding layeer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs))
embed_Weights = model_1.get_layer("embedding_1").get_weights()
embed_Weights

[array([[-0.01062573, -0.05275462,  0.00800254, ...,  0.00910792,
          0.02307833, -0.01772935],
        [ 0.01038432,  0.0319877 , -0.02703985, ..., -0.00974621,
         -0.02744362, -0.01787872],
        [ 0.02277392, -0.05065035,  0.00266097, ..., -0.01508311,
         -0.0459183 ,  0.03991572],
        ...,
        [-0.00785009, -0.04548398, -0.04408681, ...,  0.01241822,
         -0.02340556, -0.00083836],
        [-0.00659926, -0.06605855,  0.07089156, ...,  0.07888445,
         -0.0416574 ,  0.06723114],
        [ 0.11051007, -0.04001845,  0.10285289, ...,  0.05413506,
         -0.08002777,  0.10381921]], dtype=float32)]

In [102]:
embed_Weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_Weights.shape)

(10000, 128)


Now we've go the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TF has a handy tool called projector

And TF also has an incredible guide on word embeddings

In [103]:
# Create embedding  files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_Weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [104]:
# download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>